In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
import torchvision.models as models

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

dataset = datasets.ImageFolder(root="/kaggle/input/augmented-alzheimer-mri-dataset", transform=transform)

In [ ]:
train_size = int(0.8*len(dataset))
val_size = len(dataset) - train_size 

train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

In [ ]:
device = torch.device("cuda")

model = models.resnet34(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4)
model = model.to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
epochs = 5

for epoch in range(epochs):
    model.train()
    total, correct = 0, 0
    epoch_loss = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(imgs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        correct += (output.argmax(1) == labels).sum().item()

    acc = correct / len(train_ds)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {acc:.4f}")

In [ ]:
model.eval()
correct = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        correct += (outputs.argmax(1) == labels).sum().item()

val_acc = correct / len(val_ds)
print(f"Validation accuracy: {val_acc:.4f}")